<a href="https://colab.research.google.com/github/dave20874/RapidReact-Trajectory/blob/main/RapidReact_Trajectory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import math
import numpy as np

In [6]:
x = np.divide([2.4, 2.5], 2.0)
print(x)

[1.2  1.25]


In [32]:
in_to_m = 25.4/1000
flywheel_r = 2.0 * in_to_m
high_goal_h = (8*12+8) * in_to_m  # 8' 8" Goal
low_goal_h = (3*12+5) * in_to_m   # 3' 5" Goal
shooter_h = 16.0 * in_to_m        # 16" height of shooter
gravity = 9.8                     # m/s^2
m_to_ft = 1000.0/25.4/12.0
shooter_loss = 0.575
CRITICAL_ANGLE = math.radians(-30.0)
m_ball_kg = 1.0  # TODO
spin_hz = -589.0/60.0   # Shooter constant
ball_r = 9.5/2.0*in_to_m
ball_a = math.pi*ball_r*ball_r
rho = 1.225    # kg / m^3

# Extra height so ball clears the target by 6 inches.
clearance = (ball_r + 6.0) * in_to_m


def launch_v(rpm):
  v_mps = rpm/60.0*flywheel_r*2.0*math.pi  # rev/min / sec/min * m -> m/sec
  return v_mps*shooter_loss

def normalize(vector):
  mag = math.sqrt(vector[0]*vector[0] + vector[1]*vector[1])
  return np.divide(vector, mag)

# Based on https://www1.grc.nasa.gov/beginners-guide-to-aeronautics/ideal-lift-of-a-spinning-ball/
def lift(v, spin=-spin_hz):
  pi_2 = math.pi * math.pi
  b_3 = ball_r * ball_r * ball_r

  L = 4.0 / 3.0 * (4.0 * pi_2 * b_3 * spin * rho * v)
  return L


# Returns None if shot is invalid
# Returns (dist, arr_angle, max_height, rpm, trajectory) if shot is valid
def shoot(rpm, angle_rad, goal_h=high_goal_h):
  v0 = launch_v(rpm)
  # print(f"v0 = {v0}")
  vx = v0 * math.cos(angle_rad)
  vy = v0 * math.sin(angle_rad)

  x = 0.0
  y = shooter_h

  state = [x, y, vx, vy]

  h = 0.001     # 1 ms steps

  # Advance trajectory 1ms at a time until it is descending below goal height
  trajectory = []
  trajectory.append((x, y))
  report_interval = 10
  h_max = y
  cd = 0.4  # coefficient of drag

  # While traveling up or above goal height
  while (state[3] > 0.0) | (state[1] > goal_h):

    # velocity
    v = [state[2], state[3]]
    unit_v = normalize(v)
    q = 0.5 * rho * (v[0]*v[0] + v[1]*v[1])

    perp_v = [-v[1], v[0]]
    unit_perp_v = [-unit_v[1], unit_v[0]]

    f_gravity = np.multiply([0.0, -gravity], m_ball_kg)
    f_drag = np.multiply(unit_v, -cd*ball_a*q)  # TODO

    L = lift(-math.sqrt(v[0]*v[0]+v[1]*v[1]))
    f_magnus = np.multiply(unit_perp_v, L) # TODO
    f_total = np.add.reduce([f_gravity, f_drag, f_magnus])
    # print(f"f_total: {f_total}, f_magnus: {f_magnus}")
    accel = np.divide(f_total, m_ball_kg)

    # Derivative of pos is vel
    # Derivative of vel is accel
    state_dot = [state[2], state[3], accel[0], accel[1]]

    state += np.multiply(state_dot, h)

    # v_new = vy - h*gravity
    # x += vx*h
    # y += (v_new+vy)/2.0 * h
    # vy = v_new

    if state[1] > h_max:
      h_max = state[1]

    report_interval -= 1
    if report_interval == 0:
      trajectory.append((x, y))
      report_interval = 10

  arr_angle = math.atan2(state[3], state[2])

  retval = (state[0], arr_angle, h_max, rpm, trajectory)

  if h_max < goal_h + clearance:
    return None

  if arr_angle > CRITICAL_ANGLE:
    return None

  return retval

result = shoot(4000.0, math.radians(67.0))

print(f"dist:{result[0]*m_to_ft} ft, arr_ang:{math.degrees(result[1])} deg, max_h:{result[2]*m_to_ft} ft")

dist:23.713619054771364 ft, arr_ang:-45.901200505756265 deg, max_h:12.88846218236373 ft


In [33]:
for angle_deg in range(65, 90):
  angle = math.radians(angle_deg)

  min_range = None
  max_range = None
  for rpm in range(2000, 4500, 10):
    soln = shoot(rpm, angle)
    if soln is not None:
      if (min_range is None):
        min_range = soln
      elif (soln[0] < min_range[0]):
        # replace min_range solution with this solution
        min_range = soln

      if (max_range is None):
        max_range = soln
      elif (soln[0] > max_range[0]):
        # replace max_range solution with this solution
        max_range = soln

  # report min, max range solutions
  print(f"Angle: {angle_deg} deg:")
  print(f"  min: {min_range[3]} rpm, {min_range[0]*m_to_ft} ft, arr: {math.degrees(min_range[1])} deg, max_h: {min_range[2]*m_to_ft} ft")
  print(f"  max: {max_range[3]} rpm, {max_range[0]*m_to_ft} ft, arr: {math.degrees(max_range[1])} deg, max_h: {max_range[2]*m_to_ft} ft")

Angle: 65 deg:
  min: 3400 rpm, 15.995506237075032 ft, arr: -30.034161706728884 deg, max_h: 9.99653426968785 ft
  max: 4490 rpm, 29.18131473418192 ft, arr: -47.84256060510328 deg, max_h: 14.313626877196224 ft
Angle: 66 deg:
  min: 3330 rpm, 15.35557554853446 ft, arr: -30.223434161886786 deg, max_h: 9.934195631195426 ft
  max: 4490 rpm, 29.33248278338817 ft, arr: -49.319412112818426 deg, max_h: 14.638346671849325 ft
Angle: 67 deg:
  min: 3260 rpm, 14.66154819170739 ft, arr: -30.083526406976613 deg, max_h: 9.859568083658715 ft
  max: 4490 rpm, 29.457490203591554 ft, arr: -50.78834292573542 deg, max_h: 14.963700159905683 ft
Angle: 68 deg:
  min: 3200 rpm, 14.103215501339331 ft, arr: -30.370702404155942 deg, max_h: 9.813201773487332 ft
  max: 4490 rpm, 29.55615101911026 ft, arr: -52.249002103106655 deg, max_h: 15.289521522328862 ft
Angle: 69 deg:
  min: 3140 rpm, 13.511171528253213 ft, arr: -30.4508202021019 deg, max_h: 9.756036539684425 ft
  max: 4490 rpm, 29.628290072151348 ft, arr: -53.